In [249]:
# creates new directory with con image, t-maps, z-maps, and temporal BOLD 4D images
# and design files for OpenAutism (per subject, per task, per run)
#only copy over files on the spreadsheet (/om/user/rezzo/EXPERIMENT_INFO/tomloc_subject_info_internal.csv)

# FOR FUTURE / PENDING:
# once second level is run, must import one con, t-map and z-map per subject (now importing 
# one per run)

# some subjects are in more than one experiment (e.g. SAX_OA_## in both exp1
# and exp2). In these cases, all runs are copied over. There is a second option, by using the function
# identify_greater_motion, which returns a list of the old subjectids to be eliminated. Based framewise displacement. 

pilot = 1        # 1 = pilot_data / 0 = analysis_data

# import modules
from glob import glob
import re
import os.path
from itertools import repeat
import csv
from shutil import copyfile
import shutil
import random
import numpy as np
import pandas as pd
from collections import Counter

In [250]:
def Obtain_OA_list(tmp_root):
    all_OA = []
    with open(tmp_root, "r") as tsv:
        for line in csv.reader(tsv,  delimiter = ","):
            all_OA.append(line[0])
    return all_OA

In [251]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [252]:
# optional function to use, if decide to remove the experiment of duplicate subject based 
# on greater motion.

def identify_greater_motion(subject1, subject2):
    tmp_root = '/om/user/rezzo/INCLUDE_final.csv'

    with open(tmp_root, "r") as tsv:
        mot1 = []
        mot2 = []
        subjectarray = [subject1, subject2]
        readCSV = csv.reader(tsv, delimiter='\t')
        for line in readCSV:
            if subject1 == line[2]:
                mot1.append(float(line[9]))
                np.array(mot1)
            if subject2 == line[2]:
                mot2.append(float(line[9]))
                np.array(mot2)
    if np.mean(mot1) > np.mean(mot2):
        return subject1
    elif np.mean(mot2) > np.mean(mot1):
        return subject2
    else:
        return random.choice(subjectarray) # for temporarily missing data, just choose randomly

In [253]:
def Convert_2int(runs):
    new_runs = [x.strip() for x in runs.split(', ')]
    counter = -1
    for el in new_runs:
        counter = counter+1
        new_runs[counter] = int(el)
    return new_runs

In [254]:
def remove_sub(name):
    if name[:4] == 'sub-':
        name = name[4:]
    return name

In [255]:
# # Looks for subjects that participated in > 1 experiment

conversion = '/om/user/rezzo/Subject_Conversion_Table.csv'
df = pd.read_csv(conversion, header=None)

# make list of all duplicate items
a = df[0].tolist()
dup_list = [k for k,v in Counter(a).items() if v>1]

repeat_subjects = []
remove_subjects = []

for el in dup_list:
    with open(conversion, "r") as tsv:
        for line in csv.reader(tsv,  delimiter = ","):
            if el == line[0]:
                repeat_subjects.append(line[1])

list_of_duplicates = [repeat_subjects[i:i + 2] for i in range(0, len(repeat_subjects), 2)]


In [256]:
# subject look up table conversion (IGNORING undescores)
def Convert_Subname(Oldname):
    tmp_root = '/om/user/rezzo/Subject_Conversion_Table.csv'

    with open(tmp_root, "r") as tsv:
        for line in csv.reader(tsv,  delimiter = ","):
            if Oldname == line[1].replace("_",""):
                Newname = line[0]
            else:
                continue
    return Newname  

In [257]:
def Remove_Underscores(list_of_names):
    counter = 0
    for el in list_of_names:
        list_of_names[counter] = el.replace("_","")
        counter = counter + 1
    return list_of_names

In [260]:
root = '/om/group/saxelab/OpenAutism/Analysis/first_level_standard.py/'

if pilot == 1:
    OA_dir = '/om/user/rezzo/OpenAutism/pilot_data/SUBJECTS/'
    conversion = '/om/user/rezzo/EXPERIMENT_INFO/tomloc_subject_info_internal.csv'
else:
    OA_dir = '/om/user/rezzo/OpenAutism/analysis_data/SUBJECTS/'
    conversion = [] # need this file

df = pd.read_csv(conversion)
df
dup_list = []
UNIQUE_OA_IDS = df['OAID'].unique()
UNIQUE_TASKS = df['func_task'].unique()

# for each unique OA subject, return row
for newname in UNIQUE_OA_IDS:
    
    if pilot == 1:
        UNIQUE_OA_IDS = [ task for task in UNIQUE_OA_IDS if "tomloc" in task ]
    else:
        UNIQUE_OA_IDS = [ task for task in UNIQUE_OA_IDS if "tomloc" not in task ]

        #create OA directory in TIER folder is non-existent
    if (not os.path.exists(OA_dir+newname)):

        proc_streams = ['first_level_standard.py','first_level_aroma.py','first_level_gorgolewski.py']
        time_streams = ['first_level_standard_timeseries.py','first_level_aroma_timeseries.py','first_level_gorgolewski_timeseries.py']

        for stream in range(0, len(proc_streams)):

            stream_abrv = find_between(proc_streams[stream], "level_", ".py" )

            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'first_level_analyses','BOLD_data')) 
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','multivariate'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','lateralization'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','magnitude'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','interregional_cor'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','temporal_variance'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','MISC','individual_roi_masks'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','MISC','mean_roi_Temporal_Signal'))

            for task in UNIQUE_TASKS:
                # create unique rows
                dup_list = []
                dup_list.append(df.index[(df['OAID'] == newname) & (df['func_task'] == task)].tolist())
                task = Remove_Underscores([task])
                task = task[0]
                run_counter = 0
                
                for items in dup_list: # each item is the indices list of all runs for a task (across experiments)
                    site = df.iloc[items]['data_set'].astype(str)
                    site = site.tolist() #site
                    found = df.iloc[items]['SUBJID'].astype(str)
                    found = Remove_Underscores(found.tolist()) #subjid
                    
                    for el in range(0,len(items)): 
                        items_index = items[el]
                        runs = df.iloc[items_index]['BIDS_runs']
                        runs = Convert_2int(runs)
                        
                        for run in runs:
                            run_counter = run_counter+1
                            src_dir = '/om/group/saxelab/OpenAutism/Analysis/'+proc_streams[stream]+'/'+site[el]+'/sub-'+found[el]+'/'+task+'/model/run'+str(run)+'/'
                            design_dir = '/om/group/saxelab/OpenAutism/data/TextFiles/tsv_text_files/'+site[el]+'/'
                            raw_dir = "/om/group/saxelab/OpenAutism/Analysis/"+time_streams[stream]+'/'+site[el]+"/sub-"+found[el]+'/'+task+"/model/run"+str(run)+'/'
                            
                            try:
                                #con, tmap, zmap
                                prefixed1 = [filename for filename in sorted(os.listdir(src_dir)) if (filename.startswith("con_1_") and filename.endswith("_tstat.nii.gz"))]
                                copyfile(src_dir+prefixed1[0], OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/tstat1_'+task+'_'+str(run_counter)+'.nii.gz')
                                prefixed2 = [filename for filename in sorted(os.listdir(src_dir)) if (filename.startswith("con_1_") and filename.endswith("_cope.nii.gz"))]
                                copyfile(src_dir+prefixed2[0], OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/cope1_'+task+'_'+str(run_counter)+'.nii.gz')
                                prefixed3 = [filename for filename in sorted(os.listdir(src_dir)) if (filename.startswith("con_1_") and filename.endswith("_zstat.nii.gz"))]
                                copyfile(src_dir+prefixed3[0], OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/zstat1_'+task+'_'+str(run_counter)+'.nii.gz')
                            except FileNotFoundError:
                                print('WARNING: missing image file(s) in '+src_dir)
                                
                            try:
                                #temporal runs
                                prefixed4 = [filename for filename in sorted(os.listdir(raw_dir)) if (filename.startswith("res4d") and filename.endswith(".nii.gz"))]
                                copyfile(raw_dir+prefixed4[0], OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/BOLD_data/temporaldata_'+task+'_'+str(run_counter)+'.nii.gz')
                            except FileNotFoundError:
                                print('WARNING: missing temopral file(s) in '+raw_dir)
                                
                            try:
                                #design file
                                copyfile(design_dir+'sub-'+found[el]+'_task-'+task+'_run-00'+str(run)+'_events.tsv', OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/BOLD_data/design_'+task+'_'+'run-00'+str(run_counter)+'.tsv') 
                            except FileNotFoundError:
                                print('WARNING: missing design file(s) in '+design_dir)